# 410-找零兑换问题的递归解法

## 找零问题：递归解法
直接使用递归调用的解法，问题是极其低效，63分的兑换硬币问题需要进行67，716，925次递归调用

In [7]:
def recMC(coinValueList, change):
    minCoins = change
    if change in coinValueList:
        return 1 
    else:
        for i in [c for c in coinValueList if c<=change]:
            numCoins = 1 + recMC(coinValueList, change-i) 
            if numCoins < minCoins:
                minCoins = numCoins
    return minCoins

import time
t1 = time.time()
print(recMC([1,5,10,25], 63))
time.time() - t1

6


24.192595958709717

## 找零兑换问题：递归解法的改进

In [6]:
def recDC(coinValueList, change, knownResults):
    minCoins = change
    if change in coinValueList:
        knownResults[change] = 1
        return 1
    elif knownResults[change]>0:
        return knownResults[change]
    else:
        for i in [c for c in coinValueList if c<=change]:
            numCoins = 1+recDC(coinValueList, change-i, knownResults)
            if numCoins < minCoins:
                minCoins = numCoins
                knownResults[change] = minCoins
    return minCoins

import time
t1 = time.time()
print(recDC([1,5,10,25], 63,[0]*64))
time.time() - t1

6


0.0010807514190673828

# 411-找零兑换问题的动态规划解法

## 找零兑换：动态规划算法代码

In [8]:
def dpMakeChange(coinValueList, change, minCoins):
    # 从1分开始到change， 逐个计算最少硬币数
    for cents in range(1,change+1):
        # 初始化最大值等于cents 
        coinCount = cents
        # 减去每个硬币，向后查最少硬币数，同时记录总的最少记录数
        for j in [c for c in coinValueList if c<= cents]:
            if minCoins[cents-j] +1 < coinCount:
                coinCount = minCoins[cents-j] +1
        # 得到当前最少硬币数，记录到表中
        minCoins[cents] = coinCount
    return minCoins[change]

import time
t1 = time.time()
print(dpMakeChange([1,5,10,21,25], 63,[0]*64))
time.time() - t1

3


0.0010509490966796875

## 找零兑换：动态规划算法扩展代码

In [15]:
def dpMakeChange(coinValueList, change, minCoins, coinsUsed):
    for cents in range(1, change+1):
        coinCount = cents
        newCoin = 1 # 初始化新加硬币
        for j in [c for c in coinValueList if c<=cents]:
            if minCoins[cents-j]+1 <coinCount:
                coinCount = minCoins[cents-j]+1
                newCoin = j 
        minCoins[cents] = coinCount
        coinsUsed[cents] = newCoin
    return minCoins[change]


def printCoins(coinUsed, change):
    coin = change
    while coin >0:
        thisCoin = coinUsed[coin]
        print(thisCoin)
        coin -= thisCoin

amnt = 63
clist = [1,5,10,21,25]
coinUsed = [0]*(amnt+1)
coinCount = [0]*(amnt+1)
print(f'Making change for {amnt} requires')
print(dpMakeChange(clist, amnt, coinCount, coinUsed))
print('They are:')
printCoins(coinUsed,amnt)
print(coinUsed)
        

Making change for 63 requires
3
They are:
21
21
21
[0, 1, 1, 1, 1, 5, 1, 1, 1, 1, 10, 1, 1, 1, 1, 5, 1, 1, 1, 1, 10, 21, 1, 1, 1, 25, 1, 1, 1, 1, 5, 10, 1, 1, 1, 10, 1, 1, 1, 1, 5, 10, 21, 1, 1, 10, 21, 1, 1, 1, 25, 1, 10, 1, 1, 5, 10, 1, 1, 1, 10, 1, 10, 21]


In [3]:
len(coinUsed)

NameError: name 'coinUsed' is not defined

# 412-动态规划案例分析

## 博物馆大盗问题：动态规划解法

In [8]:
# 宝物的重量和价值
tr = [None, {'w':2,'v':3}, {'w':3,'v':4}
          , {'w':4,'v':8}, {'w':5,'v':8}
          , {'w':9,'v':10}]
# 大盗最大承重
max_w = 20

# 初始化二维表格m[(i,w)]
# 表示前i个宝物中， 最大重量w的组合，所得到的最大价值
# 当i什么都不取，或w上线为0时， 价值均为0 
m = {(i,w):0 for i in range(len(tr))
                 for w in range(max_w+1)}

# 逐个填写二维表格
for i in range(1, len(tr)):
    for w in range(1,max_w+1):
        if tr[i]['w']>w: # 装不下第i个宝物
            m[(i,w)] = m[i-1,w] # 不装第i个宝物
        else:
            # 不装第i个宝物， 装第i个宝物， 两种情况下取最大值
            m[(i,w)] = max(m[i-1,w],
                           m[(i-1, w-tr[i]['w'])] + tr[i]['v'])
# 输出结果 
print(m[len(tr)-1, max_w])


29


## 博物馆大盗问题：递归解法

In [10]:
# 宝物的重量和价值
tr = {(2,3),(3,4),(4,8),(5,8),(9,10)}

# 大盗最大承重
max_w = 20 

# 初始化 记忆表格m 
# key是（宝物组合，最大重量），value 是最大价值
m = {} 
def thief(tr, w):
    if tr== set() or w==0:
        m[tuple(tr), w] = 0
        return 0
    elif (tuple(tr),w) in m:
        return m[(tuple(tr),w)]
    else:
        vmax =0 
        for t in tr:
            if t[0] <= w:
                # 逐个从集合中去掉某个宝物，递归调用
                # 选出所有价值中最大值
                v = thief(tr-{t},w-t[0])+t[1]
                vmax = max(vmax,v)
        m[tuple(tr),w] = vmax
    return vmax

# 输出结果
print(thief(tr, max_w))

29
